In [1]:
import numpy as np
import json

In [2]:
def Ea(R1,R2):   # Det forventede resultat for spiller A
    return 1/(1+10**((R2-R1)/400))
def Eb(R1,R2):   # Det forventede resultat for spiller B
    return 1/(1+10**((R1-R2)/400))
def k(Mål_1,Mål_2):  # K-faktor, hvor der tages højde for mål-differensen
    return 10*abs((Mål_1-Mål_2))**(1.2)
def calc_ny_ranking(R1,R2,Mål_1,Mål_2):  # Den nye ranking for spiller A
    if Mål_1>Mål_2:
        R1_ny = R1+k(Mål_1,Mål_2)*(1-Ea(R1,R2))
        R2_ny = R2-k(Mål_1,Mål_2)*Eb(R1,R2)
    if Mål_1<Mål_2:
        R1_ny = R1-k(Mål_1,Mål_2)*Ea(R1,R2)
        R2_ny = R2+k(Mål_1,Mål_2)*(1-Eb(R1,R2))
    return int(np.round(R1_ny)), int(np.round(R2_ny))  # Returnerer de nye rankings som integers

In [6]:
# File path to store player rankings
player_rankings_file = 'player_rankings.json'

# File path to store match history
match_history_file = 'match_history.json'

# List to store match data
match_history = []

# Function to load match history from file
def load_match_history():
    global match_history
    try:
        with open(match_history_file, 'r') as file:
            match_history = json.load(file)
    except FileNotFoundError:
        # If the file doesn't exist, initialize match_history as an empty list
        match_history = []

# Function to save match history to file (limit to the most recent 10 matches)
def save_match_history():
    global match_history
    # Keep only the most recent 10 matches
    match_history = match_history[-10:]
    with open(match_history_file, 'w') as file:
        json.dump(match_history, file)

# Function to load player rankings from file
def load_player_rankings():
    try:
        with open(player_rankings_file, 'r') as file:
            player_rankings = json.load(file)
    except FileNotFoundError:
        # If the file doesn't exist, initialize player_rankings as an empty dictionary
        player_rankings = {}
    return player_rankings

# Function to save player rankings to file
def save_player_rankings(player_rankings):
    with open(player_rankings_file, 'w') as file:
        json.dump(player_rankings, file)

# Function to perform calculation, update global values, and save match data
def update_rankings():
    global match_history

    # Load player rankings from file
    player_rankings = load_player_rankings()
    
    # Load match history from file
    load_match_history()

    # Display the list of active players
    active_players = list(player_rankings.keys())
    print("Active players:", active_players)

    # Input from the user
    name1 = input("Enter the first name from the list above: ")
    name2 = input("Enter the second name from the list above: ")
    
    # Validate that the entered names are in the list
    if name1 not in active_players or name2 not in active_players:
        print("Invalid names. Please enter names from the list.")
        return
    
    # Validate that the entered names are unique
    if name1 == name2:
        print("Invalid names. Please enter unique names from the list.")
        return

    score1 = float(input("Enter the score for {} (a number between 0 and 10): ".format(name1)))
    score2 = float(input("Enter the score for {} (a number between 0 and 10): ".format(name2)))

    # Validate that the scores are not above 10 and not both are equal to 10
    if score1 > 10 or score2 > 10 or (score1 == 10 and score2 == 10):
        print("Invalid scores. Scores must be between 1 and 10, and both cannot be equal to 10.")
        return

    # Calculation function
    calculated_value1 = calc_ny_ranking(player_rankings[name1], player_rankings[name2], score1, score2)[0]
    calculated_value2 = calc_ny_ranking(player_rankings[name1], player_rankings[name2], score1, score2)[1]

    # Update player rankings
    player_rankings[name1] = calculated_value1
    player_rankings[name2] = calculated_value2

    # Save match data
    match_data = {'name1': name1, 'name2': name2, 'score1': int(score1), 'score2': int(score2)}
    match_history.append(match_data)

    # Save updated player rankings and match history to files
    save_player_rankings(player_rankings)
    save_match_history()
    
    # Print current scores of the two players
    print(f"Current ranking for {name1}: {calculated_value1}")
    print(f"Current ranking for {name2}: {calculated_value2}")

In [7]:
# Update rankings
update_rankings()

Active players: ['Aske', 'Sagar', 'Rasmus', 'Anders']
Current ranking for Aske: 910
Current ranking for Sagar: 1090
